# Functions used in getting data_list
- Adds each file in folder into data_list object. Adds the image array as well as image class based on scroll number and filename

In [1]:
def determine_class(filename):
    # A dictionary mapping substrings to their corresponding classes
    class_map = {
        "862": 0,
        
        "0026": 1,
        "0089": 1,
        "0155": 1,
        '0339': 1,
        "0353": 1,
        "0817": 1,
        '1005': 1,
        "1044": 1,
        "1413": 1,
        
        "1636": 2,
        
        '1670': 3
    }
    
    # Iterate through the dictionary to find a match
    for key, class_name in class_map.items():
        if key in filename:
            return class_name
    
    # Return a default value if no match is found
    return "unknown_class"

In [2]:
def create_zarr_arrays(directory):
    # List to store each transposed Zarr array
    data_list = []
    # Counter to track the number of files processed
    file_count = 1
    max_files = 999999 

    # Get the total number of directories (Zarr files) in the directory
    total_files = sum(1 for filename in os.listdir(directory) if os.path.isdir(os.path.join(directory, filename)))

     # Iterate through each file in the directory
    for filename in os.listdir(directory):
        # Construct the full path to the file
        file_path = os.path.join(directory, filename)
        
        # Check if it's a directory (since Zarr files are typically stored in directories)
        if os.path.isdir(file_path):
            print(f"Processing file #{file_count}/{total_files} at {os.path.basename(file_path)}")
            try:
                # Open the Zarr file
                #print(f"Opening {file_path}")
                zarr_file = zarr.open(file_path, mode='r')

                # Use Zarr attributes to retrieve the class of the image
                #tests the embedding of metadata to files when importing from globus
                if 'class' in zarr_file.attrs:
                    image_class = zarr_file.attrs['class']
                else:
                    image_class = determine_class(filename)
                    print(f"No class attribute found in {filename}; assigning class {image_class} using function.\n")

                #Create the Zarray
                if zarr_file.shape[-1] == 16:
                    #actual image data
                    zarr_array = zarr_file
                elif zarr_file.shape[0] == 16:
                    #converts (channels, h, w) to (h,w,channels)
                    zarr_array = np.transpose(zarr_file, (1,2,0))
                else:
                    print("Zarr file is not in any correct format")
        
                # Store the data along with its metadata
                data_list.append({
                    'image_data': zarr_array,  # The actual image data
                    'class': image_class,            # The class determined
                    'filename': filename             # The original filename
                })
                
                # Increment the file count
                file_count += 1
                # Stop if the maximum number of files has been processed
                if file_count >= max_files:
                    print("Reached file limit set my user. Verify variable 'max_files'")
                    break
                
            except Exception as e:
                # Handle cases where the file cannot be opened as a Zarr file
                print(f"Could not read {filename} as a Zarr file. Error: {e}")
    print("Done")
    return data_list

In [3]:
import os
import zarr
from collections import Counter
def get_data_list():
    zarr_dir = "D:\EduceLab_HDD\ImportsFromGlobus\zarrs"
    data_list = [] 
    data_list = create_zarr_arrays(zarr_dir)

    print("Created data_list succesfully. Summary of data_list: \n")    
    
    # Get all unique shapes
    unique_shapes = {item['image_data'].shape for item in data_list}
    print("Unique Shapes:")
    for shape in unique_shapes:
        print(shape)
    
    # Count the occurrences of each class
    #create a dictionary for the class count to be used for uniform subarea distribution
    class_counts = Counter(item['class'] for item in data_list)
    class_counts_dic = {}
    print("\nClass Counts:")
    for image_class, count in class_counts.items():
        class_counts_dic[image_class] = count
        print(f"{image_class}: {count}")
    print(class_counts_dic)

    return data_list

### Results from create zarrays are consistent with expectations

# Functions used in getting balanced dataset
- get_centered_boudaries, returns X boundaries of given size.
- extract_subarea, returns a slice of the image with the boundaries given
- subarea_generator, given data_list, it yields a subarea and label
- create_dataset, uses subarea_generator to create a fataset for training

In [4]:
import tensorflow as tf
import numpy as np
from collections import defaultdict

In [5]:
def get_centered_boudaries(image_shape, subarea_size, subareas_per_image):
  """
    Generate a list of bounding boxes centered around the middle of an image.

    Args:
        image_shape (tuple): A tuple representing the dimensions of the image (height, width, channels).
        subarea_size (tuple): A tuple representing the height and width of each subarea.
        subareas_per_image (int): The number of subareas to generate within the image.

    Returns:
        list: A list of tuples, each representing the bounding coordinates of a subarea within the image.
              Each tuple has the format (top_y, bottom_y, left_x, right_x).

    Functionality:
        - Calculates the center of the image using its dimensions.
        - Defines a standard deviation to control the spread of subarea locations around the center.
        - For each subarea, generates top-left coordinates based on a normal distribution centered around
          the image's midpoint.
        - Clips the coordinates to ensure each subarea is within the image boundaries.
        - Returns the list of boundaries, with each subarea centered around the middle, following a normal distribution.

    Example:
        If image_shape = (200, 200, 3), subarea_size = (50, 50), and subareas_per_image = 5,
        the function will return 5 bounding boxes focused around the center of the image, with each box
        sized 50x50 pixels.
  """

  height, width, channel = image_shape

  # Define the mean and normal distribution (center of image)
  mean_x = width // 2
  mean_y = height // 2

  # Define the standard deviation (smaller values will focus more on center)
  std_x = width // 8
  std_y = height // 8

  boundaries_list = []

  for _ in range(subareas_per_image):
    # Generate random values from the normal distribution
    top_left_x = int(np.clip(np.random.normal(loc = mean_x, scale = std_x), 0, width - subarea_size[1]))
    top_left_y = int(np.clip(np.random.normal(loc = mean_y, scale = std_y), 0, height - subarea_size[0]))
    boundaries_list.append((top_left_y, top_left_y + subarea_size[0], top_left_x, top_left_x + subarea_size[1]))
  return boundaries_list

In [6]:
def extract_subarea(image, boundaries):
  """
    Extract a subarea from an image based on given boundary coordinates.

    Args:
        image (numpy array): The input image array from which a subarea will be extracted.
                             Expected shape is (height, width, channels).
        boundaries (tuple): A tuple representing the coordinates of the subarea to extract.
                            The format should be (top_left_y, bottom_right_y, top_left_x, bottom_right_x),
                            where:
                              - top_left_y: The top boundary row index.
                              - bottom_right_y: The bottom boundary row index.
                              - top_left_x: The left boundary column index.
                              - bottom_right_x: The right boundary column index.

    Returns:
        numpy array: The extracted subarea as a slice of the original image, retaining
                     the same number of channels.

    Functionality:
        - Slices the input image array according to the specified boundaries.
        - Returns only the region within the defined boundaries, with the same number
          of color channels as the input image.

    Example:
        If boundaries = (50, 100, 30, 80), the function will return the portion of the
        image from row 50 to 100 and column 30 to 80.
    """

  top_left_y, bottom_right_y, top_left_x, bottom_right_x = boundaries
  subarea = image[top_left_y:bottom_right_y, top_left_x:bottom_right_x, :]
  return subarea

In [7]:
def subarea_generator(data_list, subarea_size=(128,128,16),subareas_per_image = 1000):
  for data_item in data_list:
    #Extract image and label
    multi_channel_image = data_item['image_data']
    image_class = data_item['class']
    #filename = data_item['filename']

    #generate centered boundaries
    boundaries_list = get_centered_boudaries(multi_channel_image.shape, subarea_size, subareas_per_image)

    #Yield each subarea and class as tensors
    for boundaries in boundaries_list:
      subarea = extract_subarea(multi_channel_image, boundaries)
      yield subarea, image_class#, filename

In [8]:
def create_dataset(data_list, subarea_size=(128,128,16),subareas_per_image = 1000):
  """
    Create a TensorFlow dataset of image subareas using a generator function.

    Args:
        data_list (list): A list containing the image data or file paths to images. Each entry corresponds to an image.
        subarea_size (tuple, optional): The desired dimensions of each subarea (height, width). Defaults to (32, 32).
        subareas_per_image (int, optional): The number of subareas to generate per image. Defaults to 10.

    Returns:
        tf.data.Dataset: A TensorFlow dataset where each element is a tuple consisting of:
                         - A tensor of shape (subarea_height, subarea_width, 16) representing
                           a subarea of the original image.
                         - An integer label associated with the subarea, of type int32.
  """
  dataset = tf.data.Dataset.from_generator(
      lambda: subarea_generator(data_list, subarea_size, subareas_per_image),
      output_signature=(
          # The 16 here is the number of channels.
          tf.TensorSpec(shape=(subarea_size[0], subarea_size[1], subarea_size[2]), dtype=tf.float32),
          tf.TensorSpec(shape=(), dtype=tf.int32)#,
          #tf.TensorSpec(shape=(), dtype=tf.string)  including filename is causing problems when feeding data into model, ignore for now
      )
  )

  return dataset

In [9]:
def get_balanced_dataset(data_list, subarea_size, size_wanted=50):
    from collections import defaultdict
    #Gets the amount of unique classes
    unique_classes = set(item['class'] for item in data_list)
    print(f"Unique classes: {unique_classes}")
    # Create a dictionary to hold data_list for each class
    data_lists = defaultdict(list)
    
    # Populate the dictionary
    for item in data_list:
        class_label = item['class']
        data_lists[class_label].append(item)

    #Convert back to lists of lists
    data_lists_as_list = [data_lists[class_label] for class_label in sorted(unique_classes)]
    print(f"Number of class-specific lists: {len(data_lists_as_list)}")    
    xs = []
    for data_list in data_lists_as_list:  # Iterate over each class-specific data list
        x = (size_wanted // 10) / len(data_list)  # Compute x for the current data list
        x = round(x * 10)
        xs.append(x)
        
    datasets = []  # List to store all created datasets
    for data_list, x in zip(data_lists_as_list, xs):
        dataset = create_dataset(data_list, subarea_size=subarea_size, subareas_per_image=x)
        if sum(1 for _ in dataset) > 0:
            datasets.append(dataset)
        else:
            print(f"Warning: Empty dataset for class {data_list[0]['class']}.")

# Inspect datasets
    for i, dataset in enumerate(datasets):
        print(f"Inspecting Dataset {i}:")
        dataset_size = sum(1 for _ in dataset)
        print(f"Dataset {i} size: {dataset_size}")
        for image, label in dataset.take(1):  # Inspect the first sample
            print(f"  Image shape: {image.shape}, Label: {label}")

    print("Finished creating datasets.")
    
    return datasets

# Functions used to split into training and validation datasets

In [10]:
from collections import Counter

def get_class_counts(dataset):
    """Count occurrences of each class in the dataset."""
    class_counts = Counter()
    for _, labels in dataset.unbatch():  # Unbatch to access individual samples
        class_counts[int(labels.numpy())] += 1
    return class_counts

In [11]:
def split_train_validation(datasets, split_ratio=0.8):
    train_datasets = []
    validation_datasets = []
    
    for dataset in datasets:
        # Count total samples in the dataset
        total_samples = sum(1 for _ in dataset)
        
        # Compute split index
        split_index = int(total_samples * split_ratio)
        
        # Split the dataset
        train_dataset = dataset.take(split_index)  # First `split_index` samples
        validation_dataset = dataset.skip(split_index)  # Remaining samples
        
        # Append to respective lists
        train_datasets.append(train_dataset)
        validation_datasets.append(validation_dataset)
    
    return train_datasets, validation_datasets

In [12]:
def train_val_split(dataset, batch_size, split_ratio):
    train_datasets, validation_datasets = split_train_validation(dataset, split_ratio)
    # Shuffle and batch
    train_datasets = [train.shuffle(buffer_size=100).batch(batch_size=batch_size) for train in train_datasets]
    validation_datasets = [val.batch(batch_size=batch_size) for val in validation_datasets]
    
    # Combine into unified datasets
    full_train_dataset = train_datasets[0]
    for train in train_datasets[1:]:
        full_train_dataset = full_train_dataset.concatenate(train)
    
    full_validation_dataset = validation_datasets[0]
    for val in validation_datasets[1:]:
        full_validation_dataset = full_validation_dataset.concatenate(val)

    # Calculate and print sizes
    train_size = sum(1 for _ in full_train_dataset)
    val_size = sum(1 for _ in full_validation_dataset)
    print(f"Final train dataset size: {train_size} batches")
    print(f"Final validation dataset size: {val_size} batches")

    # Get class counts
    train_class_counts = get_class_counts(full_train_dataset)
    val_class_counts = get_class_counts(full_validation_dataset)

    print("Class counts in training dataset:", train_class_counts)
    print("Class counts in validation dataset:", val_class_counts)

    return full_train_dataset, full_validation_dataset

# Model

In [13]:
def get_model(subarea_size = (128,128,16)):
    #Define a simple CNN model for demo
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(subarea_size[0], subarea_size[1], subarea_size[2])),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(4, activation='softmax')  # 4 classes
    ])
    return model

In [14]:
def train(model, train_data, validation_data, epochs):
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data,
                        epochs=epochs,
                        validation_data = validation_data)
    return history

In [15]:
import matplotlib.pyplot as plt

In [16]:
def visualize_history(history):
    # Plot accuracy
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Running

In [17]:
data_list = get_data_list()

Processing file #1/65 at PHerc0026Cr01.zarr
No class attribute found in PHerc0026Cr01.zarr; assigning class 1 using function.

Processing file #2/65 at PHerc0026Cr02.zarr
No class attribute found in PHerc0026Cr02.zarr; assigning class 1 using function.

Processing file #3/65 at PHerc0026Cr03.zarr
No class attribute found in PHerc0026Cr03.zarr; assigning class 1 using function.

Processing file #4/65 at PHerc0026Cr04.zarr
No class attribute found in PHerc0026Cr04.zarr; assigning class 1 using function.

Processing file #5/65 at PHerc0026Cr05.zarr
No class attribute found in PHerc0026Cr05.zarr; assigning class 1 using function.

Processing file #6/65 at PHerc0026Cr06.zarr
No class attribute found in PHerc0026Cr06.zarr; assigning class 1 using function.

Processing file #7/65 at PHerc0026Cr07.zarr
No class attribute found in PHerc0026Cr07.zarr; assigning class 1 using function.

Processing file #8/65 at PHerc0089Cr01.zarr
No class attribute found in PHerc0089Cr01.zarr; assigning class 1 u

In [18]:
subarea_size = (128,128,16)
num_of_subareas = 100
dataset = get_balanced_dataset(data_list, subarea_size,  num_of_subareas)

Unique classes: {0, 1, 2, 3}
Number of class-specific lists: 4
Inspecting Dataset 0:
Dataset 0 size: 102
  Image shape: (128, 128, 16), Label: 0
Inspecting Dataset 1:
Dataset 1 size: 110
  Image shape: (128, 128, 16), Label: 1
Inspecting Dataset 2:
Dataset 2 size: 100
  Image shape: (128, 128, 16), Label: 2
Inspecting Dataset 3:
Dataset 3 size: 99
  Image shape: (128, 128, 16), Label: 3
Finished creating datasets.


In [19]:
batch_size = 16
split_ratio = 0.8

training_data, validation_data = train_val_split(dataset, batch_size, split_ratio)

Final train dataset size: 22 batches
Final validation dataset size: 8 batches
Class counts in training dataset: Counter({1: 88, 0: 81, 2: 80, 3: 79})
Class counts in validation dataset: Counter({1: 22, 0: 21, 2: 20, 3: 20})


In [20]:
model = get_model()

In [21]:
epochs = 5
history = train(model, training_data, validation_data, epochs)

Epoch 1/5
22/22 [==============================] - 28s 1s/step - loss: 2.8601 - accuracy: 0.1982 - val_loss: 1.3870 - val_accuracy: 0.2410
Epoch 2/5
22/22 [==============================] - 29s 1s/step - loss: 1.4397 - accuracy: 0.0518 - val_loss: 1.3860 - val_accuracy: 0.2651
Epoch 3/5
22/22 [==============================] - 31s 1s/step - loss: 1.3891 - accuracy: 0.2683 - val_loss: 1.3860 - val_accuracy: 0.2651
Epoch 4/5
22/22 [==============================] - 29s 1s/step - loss: 1.3885 - accuracy: 0.2683 - val_loss: 1.3859 - val_accuracy: 0.2651
Epoch 5/5
22/22 [==============================] - 29s 1s/step - loss: 1.3883 - accuracy: 0.2683 - val_loss: 1.3859 - val_accuracy: 0.2651


In [ ]:
visualize_history(history)

# Model testing

In [ ]:
import numpy as np
import tensorflow as tf

# Function to aggregate predictions using majority voting or average probabilities
def aggregate_predictions(predictions, method='majority'):
    if method == 'majority':
        # Use majority vote across subareas for final class
        final_prediction = np.argmax(np.bincount(predictions))
    elif method == 'average':
        # Average the class probabilities across subareas
        avg_probabilities = np.mean(predictions, axis=0)

        # Returns the index of highest class probability
        final_prediction = np.argmax(avg_probabilities)
    return final_prediction


In [ ]:
method = 'average'

final_predictions = []
true_labels = [item['class'] for item in test_data]  # Ground truth labels

# Process each image in the test_data individually
for item in test_data:
    # Generate subareas for this single image
    subarea_dataset = create_dataset([item], subarea_size=subarea_size, subareas_per_image = 100)
    subarea_dataset = subarea_dataset.map(lambda x, y: x)  # Keep only the image data
    subarea_dataset = subarea_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    # Collect predictions for all subareas of this image
    subarea_predictions = []
    for subareas in subarea_dataset:
        preds = model.predict(subareas)
        subarea_predictions.extend(preds)  # Collect all subarea predictions

    # Aggregate predictions for this image
    subarea_predictions = np.array(subarea_predictions)
    
    if method == 'majority':
        # Get class prediction for each subarea and apply majority voting
        subarea_classes = np.argmax(subarea_predictions, axis=1)
        final_class = aggregate_predictions(subarea_classes, method='majority')
    elif method == 'average':
        # Average class probabilities and choose the class with highest average
        final_class = aggregate_predictions(subarea_predictions, method='average')

    # Append the final prediction for this image
    final_predictions.append(final_class)

# Calculate the accuracy on the full images
accuracy = np.mean(np.array(final_predictions) == np.array(true_labels))
print(f"Final Test Accuracy: {accuracy}")

In [ ]:
print(subarea_predictions)

In [ ]:
print(final_predictions)

In [ ]:
true_labels = [item['class'] for item in test_data]
print(true_labels)

In [ ]:
# Aggregate the probabilities by averaging across subareas
avg_probabilities = np.mean(subarea_predictions, axis=0)
print("Averaged probabilities across subareas:", avg_probabilities)

# Get the predicted class from the averaged probabilities
final_prediction_index = np.argmax(avg_probabilities)
predicted_class = final_prediction_index + 1  # Convert our 1-based class label

print(f"Predicted class (in our labeling system): {predicted_class}")
